## Демонстрация пайплайна обработки PDF-документов

**Цель:** Показать полный цикл обработки PDF-файла - от извлечения текста через OCR до структурирования данных в JSON.

## Архитектура проекта

```
PDF → Конвертация в изображения → Предобработка → OCR → Извлечение данных → JSON
```
#### В телеграм-боте

```
Команда /start → Главное меню
↓
Кнопка "Скан" → Ожидание PDF
↓
[Telegram Bot]
↓ (PDF файл)
[FastAPI Server] -> проверка на формат
↓
PDF → Конвертация в изображения → Предобработка → OCR → Извлечение данных → JSON
↓
[Telegram Bot]
↓
Отображение результата пользователю
```

Установка необходимых библиотек

In [2]:
%pip install pymupdf easyocr opencv-python fastapi uvicorn python-multipart aiogram ipykernel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.2/698.2 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.13.2
    Uninstalling aiohttp-3.13.2:
      Successfully uninstalled aiohttp-3.13.2


Импорт библиотек

In [3]:
import fitz
import easyocr
import cv2
import numpy as np
import re
import os
import json
from PIL import Image
import matplotlib.pyplot as plt

Функции предобработки PDF

In [4]:
def pdf_to_png(pdf_path, output_dir="temp_pages", dpi=350):
    os.makedirs(output_dir, exist_ok=True)

    doc = fitz.open(pdf_path)
    image_paths = []

    for i, page in enumerate(doc):
        mat = fitz.Matrix(dpi/72, dpi/72)
        pix = page.get_pixmap(matrix=mat)
        out_path = os.path.join(output_dir, f"page_{i+1}.png")
        pix.save(out_path)
        image_paths.append(out_path)

    doc.close()
    return image_paths

def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, None, fx=1.3, fy=1.3, interpolation=cv2.INTER_LINEAR)
    img = cv2.GaussianBlur(img, (3,3), 0)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)[1]
    out = img_path.replace(".png", "_prep.png")
    cv2.imwrite(out, img)
    return out

def clean_text(text: str):
    text = text.replace("\xa0", " ")
    text = re.sub(r"[ ]{2,}", " ", text)
    return text

Инициализация OCR

In [5]:
reader = easyocr.Reader(['ru', 'en'], gpu=False)

def ocr_image(image_path):
    image_path = preprocess_image(image_path)
    results = reader.readtext(image_path, detail=0, paragraph=True)
    return "\n".join(results)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

Функции извлечения данных

In [6]:
def extract_document_type(text):
    patterns = [
        r"сч[её]т[\s\-]*оферта",
        r"сч[её]т[\s\-]*договор",
        r"сч[её]т[\s\-]*на[\s\-]*оплату",
        r"акт[\s\-]*выполненных[\s\-]*работ",
        r"сч[её]т"
    ]
    for p in patterns:
        m = re.search(p, text, flags=re.IGNORECASE)
        if m:
            return m.group(0).strip().title()
    return None

def extract_document_number(text):
    m = re.search(
        r"(?:сч[её]т[^\n]{0,20}?№?\s*)([A-Za-zА-Яа-я0-9\-/]+)",
        text,
        flags=re.IGNORECASE
    )
    if m:
        return m.group(1)

    m = re.search(r"№\s*([A-Za-zА-Яа-я0-9\-/]+)", text)
    return m.group(1) if m else None


def extract_document_date(text):
    m = re.search(r"(\d{1,2}[.\-/]\d{1,2}[.\-/]\d{2,4})", text)
    if m:
        return m.group(1)

    month_map = {
        "января": "01", "февраля": "02", "марта": "03", "апреля": "04",
        "мая": "05", "июня": "06", "июля": "07", "августа": "08",
        "сентября": "09", "октября": "10", "ноября": "11", "декабря": "12"
    }

    m = re.search(
        r"(\d{1,2})\s+(января|февраля|марта|апреля|мая|июня|июля|августа|"
        r"сентября|октября|ноября|декабря)\s+(\d{4})",
        text,
        flags=re.IGNORECASE
    )

    if m:
        day = m.group(1).zfill(2)
        month = month_map[m.group(2).lower()]
        year = m.group(3)
        return f"{day}.{month}.{year}"

    m = re.search(
        r"(\d{1,2})\s+(января|февраля|марта|апреля|мая|июня|июля|августа|"
        r"сентября|октября|ноября|декабря)\s+(\d{4})\s*г\.?",
        text,
        flags=re.IGNORECASE
    )

    if m:
        day = m.group(1).zfill(2)
        month = month_map[m.group(2).lower()]
        year = m.group(3)
        return f"{day}.{month}.{year}"

    return None


def extract_inn(text):
    m = re.search(r"ИНН\s*/\s*КПП\s*([0-9]{10,12})", text, flags=re.IGNORECASE)
    if m: return m.group(1)

    m = re.search(r"ИНН[: ]*([0-9]{10,12})", text, flags=re.IGNORECASE)
    return m.group(1) if m else None

def extract_kpp(text):
    m = re.search(r"ИНН\s*/\s*КПП\s*[0-9]{10,12}\s*/\s*([0-9]{9})", text, re.IGNORECASE)
    if m: return m.group(1)

    m = re.search(r"КПП[: ]*([0-9]{9})", text, re.IGNORECASE)
    return m.group(1) if m else None

def extract_bik(text):
    m = re.search(r"БИК[: ]*([0-9]{9})", text, re.IGNORECASE)
    return m.group(1) if m else None

def extract_account(text):
    m = re.search(r"(?:р[\/ ]?с|расчетный счет)[^\d]*([0-9]{20})", text, re.IGNORECASE)
    return m.group(1) if m else None

def extract_corr_account(text):
    m = re.search(r"(?:кор[\/ ]?сч|к\/с)[^\d]*([0-9]{20})", text, re.IGNORECASE)
    return m.group(1) if m else None

def extract_total_amount(text):
    m = re.search(
        r"(?:итого|к оплате)[^0-9]*([\d\s.,]+ ?(?:руб|₽)?)",
        text,
        re.IGNORECASE
    )
    return m.group(1).strip() if m else None

def extract_vat_amount(text):
    m = re.search(
        r"(?:НДС|в т\.ч\. НДС)[^0-9]*([\d\s.,]+ ?(?:руб|₽)?)",
        text,
        re.IGNORECASE
    )
    return m.group(1).strip() if m else None

def extract_vat_rate(text):
    m = re.search(r"НДС\s*\(?\s*(\d{1,2}%|без НДС)\s*\)?", text, re.IGNORECASE)
    return m.group(1) if m else None

def extract_items(text):
    items = []
    lines = text.split("\n")

    for line in lines:
        if re.search(r"\d+[,\.]?\d*\s*(руб|₽)", line, re.IGNORECASE):
            items.append({"raw": line.strip()})

    return items


Функция построения JSON

In [7]:
def build_json(text):
    return {
        "document_type": extract_document_type(text),
        "document_number": extract_document_number(text),
        "document_date": extract_document_date(text),

        "supplier": {
            "name": None,
            "inn": extract_inn(text),
            "kpp": extract_kpp(text),
            "address": None,
            "bank": None,
            "bik": extract_bik(text),
            "account": extract_account(text),
            "correspondent_account": extract_corr_account(text)
        },

        "buyer": {
            "name": None,
            "inn": None,
            "kpp": None,
            "address": None
        },

        "total_amount": extract_total_amount(text),
        "vat_amount": extract_vat_amount(text),
        "vat_rate": extract_vat_rate(text),
        "amount_words": None,

        "items": extract_items(text),

        "payment_purpose": None,
        "payment_deadline": None
    }

Основная обработка PDF-файла

In [8]:
def process_pdf(pdf_path):
    pages = pdf_to_png(pdf_path)
    full_text = ""

    for img in pages:
        text = ocr_image(img)
        full_text += "\n" + clean_text(text)

    return build_json(full_text)

Демонстрация работы на примере счет.pdf

In [9]:
try:
    result = process_pdf("/content/счет.pdf")

    print(json.dumps(result, ensure_ascii=False, indent=2))

    print(f"Тип документа: {result['document_type']}")
    print(f"Номер документа: {result['document_number']}")
    print(f"Дата документа: {result['document_date']}")
    print(f"ИНН поставщика: {result['supplier']['inn']}")
    print(f"КПП поставщика: {result['supplier']['kpp']}")
    print(f"БИК банка: {result['supplier']['bik']}")
    print(f"Расчетный счет: {result['supplier']['account']}")
    print(f"Корреспондентский счет: {result['supplier']['correspondent_account']}")
    print(f"Общая сумма: {result['total_amount']}")
    print(f"Сумма НДС: {result['vat_amount']}")
    print(f"Ставка НДС: {result['vat_rate']}")
    print(f"Количество позиций: {len(result['items'])}")

    pages = pdf_to_png("счет.pdf")
    print(f"Создано изображений страниц: {len(pages)}")

    for i, page in enumerate(pages):
        prep_img = preprocess_image(page)
        print(f"Страница {i+1}: оригинал - {page}, обработанная - {prep_img}")

except Exception as e:
    print(f"Ошибка при обработке: {e}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{
  "document_type": "Счет-Оферта",
  "document_number": "у-оферте",
  "document_date": "49-84-45",
  "supplier": {
    "name": null,
    "inn": "1831096455",
    "kpp": "184101001",
    "address": null,
    "bank": null,
    "bik": "044525974",
    "account": null,
    "correspondent_account": "30101810145250000974"
  },
  "buyer": {
    "name": null,
    "inn": null,
    "kpp": null,
    "address": null
  },
  "total_amount": "191 468,22 руб",
  "vat_amount": "31 911,37 руб",
  "vat_rate": "20%",
  "amount_words": null,
  "items": [
    {
      "raw": "(ОМПОНЕнтЫ высОKИЖ ТЕХНОЛОГИЙ литан ДЛЯ РОССИЙСКОЙ ПРОМЫШЛЕННОСТИ Компания Элитан(ТМ) с 1999 года на промышленном рынке России Международный бизнес-рейтинг =A1 от финансового рейтингового агентства Dun & Bradstreet (США): Крупнейший российский дистрибьютор компонентов в списке ТОП-25 аналитического 'Центра Современной Электроники\" wwwsovel.org Продавец: ООО \"Элитан Трейд\" 426028, Удмуртская Республика, г Ижевск; ул Областная зд.5Б к